# Introduction à Othello-GPT

Ceci est un notebook de démonstration qui charge les poids du modèle **Othello-GPT** issus de l’article [*Emergent World Representations*](https://arxiv.org/pdf/2210.13382.pdf) vers le package **TransformerLens**. Vous pouvez consulter le billet de blog [Do Large Language Models learn world models or just surface statistics?](https://thegradient.pub/othello/), l’[article original sur arxiv](https://arxiv.org/pdf/2210.13382.pdf) ainsi que le github [github](https://github.com/likenneth/othello_world/) correspondant.

Dans ce notebook nous allons voir comment faire du retro-engineering sur ce modèle. Notre but sera de répondre aux questions suivantes :

* Pourquoi les **probes non linéaires** fonctionnent nettement mieux que les probes linéaires ?
  * Le modèle représente-t-il le plateau en interne d’une manière exploitable mais **non linéaire** ?
  * Existe-t-il des représentations plus simples (p. ex. diagonales du plateau, nombre de pions noirs, qu’une case soit vide) qu’un probe non-linéaire utiliserait pour reconstituer les positions, alors que le modèle lui raisonnerait surtout dans cet espace latent plus simple ?
* Qu'est-ce qu'implique exactement le **model editing** (modification du modèle) :
  * Dans l'article, les auteurs interviennent sur de nombreuses couches à la fois. Quelle est l’**édition minimale** qui fonctionne ?
    * Peut-on éditer juste avant la couche finale ?
    * Peut-on faire **une seule** édition plutôt qu’à travers plusieurs couches ?
  * Si l’on compare les activations **avant/après** l’édition, qu’est-ce qui change ?
    * Quelles composantes déplacent leur sortie et comment cela affecte-t-il les logits ?
    * Y a-t-il une **profondeur de composition** significative, ou cela n’affecte-t-il que les logits de sortie ?
* Peut-on trouver des **circuits non triviaux** dans le modèle ?
  * Commencer par des techniques **exploratoires** (p. ex. attribution directe des logits, inspection des patterns des attention heads).
  * Choisir une sous-tâche simple (p. ex. déterminer si une case est vide) et tenter de l’interpréter.

J’ai téléchargé des **checkpoints** sur HuggingFace, téléchargeables automatiquement, et un extrait de code. La section d’installation montre comment faire.

Si vous préférez utiliser le **code donné par les auteurs**, j’ai écrit un script pour charger et convertir leurs checkpoints — il se trouve plus bas.

Pour démarrer, voyez le **[tutoriel principal de TransformerLens](https://neelnanda.io/transformer-lens-demo)** et le **[tutoriel sur les techniques exploratoires](https://neelnanda.io/exploratory-analysis-demo)**, ainsi que l’**excellent GitHub** des auteurs ([ot**hello world**](https://github.com/likenneth/othello_world/)) avec divers notebooks montrant comment charger les entrées, etc. Enfin, jetez un œil à ma série **[Concrete Open Problems in Mechanistic Interpretability](https://www.lesswrong.com/s/yivyHaCAmMJ3CqSyj)** — notamment le billet sur les problèmes algorithmiques — pour des conseils sur ce style de recherche.


## Mise en place

Active le rechargement automatique des modules modifiés dans un notebook Jupyter.
Cela permet de modifier un fichier Python externe (par ex. une librairie en cours d'édition),
puis de réexécuter une cellule sans avoir à redémarrer le kernel pour que les changements soient pris en compte.

In [1]:
from IPython import get_ipython
ipython = get_ipython()
ipython.run_line_magic("load_ext", "autoreload")
ipython.run_line_magic("autoreload", "2")

On installe les packages nécessaires: transformer_lens, circuitsvis et torchtyping.

In [2]:
%pip install transformer_lens
%pip install circuitsvis
%pip install torchtyping

  Using cached typeguard-4.4.4-py3-none-any.whl.metadata (3.3 kB)
Using cached typeguard-4.4.4-py3-none-any.whl (34 kB)
  Attempting uninstall: typeguard
    Found existing installation: typeguard 2.13.3
    Uninstalling typeguard-2.13.3:
      Successfully uninstalled typeguard-2.13.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtyping 0.1.5 requires typeguard<3,>=2.11.1, but you have typeguard 4.4.4 which is incompatible.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Using cached typeguard-2.13.3-py3-none-any.whl.metadata (3.6 kB)
Using cached typeguard-2.13.3-py3-none-any.whl (17 kB)
  Attempting uninstall: typeguard
    Found existing installation: typeguard 4.4.4
    Uninstalling typeguard-4.4.4:
      Successfully uninstalled typeguard-4.4.4
ERROR: pip's depend

Ce code configure l’outil de visualisation Plotly pour qu’il affiche correctement les graphiques dans un environnement de type notebook (Jupyter ou VSCode). Par défaut, Plotly propose plusieurs moteurs de rendu (appelés renderers), et certains fonctionnent mieux selon l’interface utilisée (navigateur, Colab, notebook local, etc.). Ici, on force l’utilisation du renderer "notebook_connected", qui permet d’intégrer les graphiques directement dans le notebook et de garder l’interactivité. La dernière ligne affiche dans la console le renderer effectivement choisi, afin de vérifier que la configuration a bien été appliquée.

In [3]:
# Plotly needs a different renderer for VSCode/Notebooks vs Colab argh
import plotly.io as pio
pio.renderers.default = "notebook_connected"
print(f"Using renderer: {pio.renderers.default}")

Using renderer: notebook_connected


In [6]:
import circuitsvis as cv

# Testing that the library works
cv.examples.hello("Artificialis")

In [8]:
# Import stuff
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import einops
from fancy_einsum import einsum
import tqdm.auto as tqdm
import random
from pathlib import Path
import plotly.express as px
from torch.utils.data import DataLoader

from torchtyping import TensorType as TT
from typing import List, Union, Optional
from functools import partial
import copy

import itertools
from transformers import AutoModelForCausalLM, AutoConfig, AutoTokenizer
import dataclasses
import datasets
from IPython.display import HTML

In [9]:
import transformer_lens
import transformer_lens.utils as utils
from transformer_lens.hook_points import (
    HookedRootModule,
    HookPoint,
)  # Hooking utilities
from transformer_lens import (
    HookedTransformer,
    HookedTransformerConfig,
    FactoredMatrix,
    ActivationCache,
)

In [10]:
torch.set_grad_enabled(False)

torch.autograd.grad_mode.set_grad_enabled(mode=False)

In [11]:
def imshow(tensor, renderer=None, xaxis="", yaxis="", **kwargs):
    px.imshow(
        utils.to_numpy(tensor),
        color_continuous_midpoint=0.0,
        color_continuous_scale="RdBu",
        labels={"x": xaxis, "y": yaxis},
        **kwargs,
    ).show(renderer)


def line(tensor, renderer=None, xaxis="", yaxis="", **kwargs):
    px.line(utils.to_numpy(tensor), labels={"x": xaxis, "y": yaxis}, **kwargs).show(
        renderer
    )


def scatter(x, y, xaxis="", yaxis="", caxis="", renderer=None, **kwargs):
    x = utils.to_numpy(x)
    y = utils.to_numpy(y)
    px.scatter(
        y=y, x=x, labels={"x": xaxis, "y": yaxis, "color": caxis}, **kwargs
    ).show(renderer)

## Othello-GPT

In [12]:
LOAD_AND_CONVERT_CHECKPOINT = False

In [13]:
import transformer_lens.utils as utils

cfg = HookedTransformerConfig(
    n_layers=8,
    d_model=512,
    d_head=64,
    n_heads=8,
    d_mlp=2048,
    d_vocab=61,
    n_ctx=59,
    act_fn="gelu",
    normalization_type="LNPre",
)
model = HookedTransformer(cfg)

In [14]:
# NBVAL_IGNORE_OUTPUT
sd = utils.download_file_from_hf(
    "NeelNanda/Othello-GPT-Transformer-Lens",
    "synthetic_model.pth"
)
# champion_ship_sd = utils.download_file_from_hf("NeelNanda/Othello-GPT-Transformer-Lens", "championship_model.pth")
model.load_state_dict(sd)

<All keys matched successfully>

In [15]:
# An example input
sample_input = torch.tensor(
    [
        [
            20,
            19,
            18,
            10,
            2,
            1,
            27,
            3,
            41,
            42,
            34,
            12,
            4,
            40,
            11,
            29,
            43,
            13,
            48,
            56,
            33,
            39,
            22,
            44,
            24,
            5,
            46,
            6,
            32,
            36,
            51,
            58,
            52,
            60,
            21,
            53,
            26,
            31,
            37,
            9,
            25,
            38,
            23,
            50,
            45,
            17,
            47,
            28,
            35,
            30,
            54,
            16,
            59,
            49,
            57,
            14,
            15,
            55,
            7,
        ]
    ]
)

model(sample_input).argmax(dim=-1)

tensor([[21, 41, 40, 34, 40, 41,  3, 11, 21, 43, 40, 21, 28, 50, 33, 50, 33,  5,
         33,  5, 52, 46, 14, 46, 14, 47, 38, 57, 36, 50, 38, 15, 28, 26, 28, 59,
         50, 28, 14, 28, 28, 28, 28, 45, 28, 35, 15, 14, 30, 59, 49, 59, 15, 15,
         14, 15,  8,  7,  8]], device='cuda:0')

In [16]:
# Charge Othello-GPT (poids déjà portés depuis l’article Emergent World Representations)
model = HookedTransformer.from_pretrained("othello-gpt")

Loaded pretrained model othello-gpt into HookedTransformer


In [19]:
model

HookedTransformer(
  (embed): Embed()
  (hook_embed): HookPoint()
  (pos_embed): PosEmbed()
  (hook_pos_embed): HookPoint()
  (blocks): ModuleList(
    (0-7): 8 x TransformerBlock(
      (ln1): LayerNormPre(
        (hook_scale): HookPoint()
        (hook_normalized): HookPoint()
      )
      (ln2): LayerNormPre(
        (hook_scale): HookPoint()
        (hook_normalized): HookPoint()
      )
      (attn): Attention(
        (hook_k): HookPoint()
        (hook_q): HookPoint()
        (hook_v): HookPoint()
        (hook_z): HookPoint()
        (hook_attn_scores): HookPoint()
        (hook_pattern): HookPoint()
        (hook_result): HookPoint()
      )
      (mlp): MLP(
        (hook_pre): HookPoint()
        (hook_post): HookPoint()
      )
      (hook_attn_in): HookPoint()
      (hook_q_input): HookPoint()
      (hook_k_input): HookPoint()
      (hook_v_input): HookPoint()
      (hook_mlp_in): HookPoint()
      (hook_attn_out): HookPoint()
      (hook_mlp_out): HookPoint()
      (hoo

In [22]:
model.all_head_labels()

['L0H0',
 'L0H1',
 'L0H2',
 'L0H3',
 'L0H4',
 'L0H5',
 'L0H6',
 'L0H7',
 'L1H0',
 'L1H1',
 'L1H2',
 'L1H3',
 'L1H4',
 'L1H5',
 'L1H6',
 'L1H7',
 'L2H0',
 'L2H1',
 'L2H2',
 'L2H3',
 'L2H4',
 'L2H5',
 'L2H6',
 'L2H7',
 'L3H0',
 'L3H1',
 'L3H2',
 'L3H3',
 'L3H4',
 'L3H5',
 'L3H6',
 'L3H7',
 'L4H0',
 'L4H1',
 'L4H2',
 'L4H3',
 'L4H4',
 'L4H5',
 'L4H6',
 'L4H7',
 'L5H0',
 'L5H1',
 'L5H2',
 'L5H3',
 'L5H4',
 'L5H5',
 'L5H6',
 'L5H7',
 'L6H0',
 'L6H1',
 'L6H2',
 'L6H3',
 'L6H4',
 'L6H5',
 'L6H6',
 'L6H7',
 'L7H0',
 'L7H1',
 'L7H2',
 'L7H3',
 'L7H4',
 'L7H5',
 'L7H6',
 'L7H7']

In [24]:
total_params = sum(p.numel() for p in model.parameters())
print(f"Nombre total de paramètres : {total_params:,}")

Nombre total de paramètres : 25,295,421
